In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
from collections import OrderedDict

### Data Load

1572 to 1339

In [2]:
# movie detail load
df_detail = pd.read_json('IMDB_movie_details.json', lines=True)

df_detail['plot_synopsis'].replace('', np.nan, inplace=True)
df_detail.dropna(subset=['plot_synopsis'], inplace=True)

df_detail.head()

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...


In [3]:
# synopsis sentence split
synopsis = df_detail[['movie_id','plot_synopsis']]

sentences = []
for row in synopsis.itertuples():
    for sentence in sent_tokenize(row[2]):
        sentences.append((row[1], sentence))
new_df = pd.DataFrame(sentences, columns=['ID', 'SENTENCE'])

In [4]:
# create count dummy
new_df['count'] = new_df.groupby(new_df['ID']).transform('count')

# group by
gr = new_df.groupby(['ID','SENTENCE']).count()
ggr = gr.reset_index()

# cumulative sum
ggr['cumsum'] = ggr.groupby('ID')['count'].transform(pd.Series.cumsum)

# sentence count
ggr['total'] = ggr.groupby(ggr['ID']).transform('count').iloc[:,0]

# criterion (50%)
ggr['criterion'] = ggr['total']*0.5

# extract synopsis
df_sentence = ggr[ggr['cumsum'] > ggr['criterion']]

# sentence to document
df_sentence['plot_synopsis'] = df_sentence.groupby(['ID'])['SENTENCE'].transform(lambda x : ' '.join(x))
df_sentence = df_sentence.iloc[:,[0,6]]

# to dataframe
df_doc = df_sentence.drop_duplicates()

C:\Users\User\anaconda3\envs\juunho\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
df_doc

,ID,plot_synopsis
24,tt0015864,"Larsen tries to eject the LP, but another ""lon..."
58,tt0017136,Rotwang complies out of ulterior motives: he k...
153,tt0017925,Johnnie can't believe it. Johnnie directs anot...
300,tt0018455,The Man refuses to look his wife in the eye as...
371,tt0019254,"One or two of them, believing that she is inde..."
...,...,...
103260,tt5439796,"Jimmy puts out the fire and yells ""Cauliflower..."
103321,tt5519340,She tells them that the Shield of Light can he...
103392,tt5580036,"She continuously stumbles on the ice, but Tony..."
103457,tt5580390,No rankings. She and Giles take the Asset away...
